## 0. AutoML 실험 결과 및 분석

### 0.1 AutoML 파이프라인 및 Search Space

본 과제에서는 세 가지 회귀 데이터셋(Airfoil, Concrete, Abalone)에 대해 비교적 최신 auto-sklearn2 라이브러리를 활용한 AutoML 파이프라인을 구현했습니다. 파이프라인은 다음과 같이 구성되었습니다:

1. **데이터 전처리**:
   - 범주형 변수(Abalone 데이터셋의 'Sex' 열)에 대한 원-핫 인코딩

2. **Search Space**:
    AutoML을 기반으로 하기에 이에 내제된 모든 Search Space가 적용됨
   - **모델 탐색**: 랜덤 포레스트, 그래디언트 부스팅, 선형 회귀, 서포트 벡터 회귀 등 다양한 회귀 모델
   - **스케일링 방법**: StandardScaler, MinMaxScaler, RobustScaler
   - **하이퍼파라미터**: 트리 깊이, 학습률, 정규화 강도 등 각 모델별 주요 파라미터
   - **탐색 시간 제한**: 120초 (각 데이터셋당)

### 0.2 데이터셋별 최적 모델 및 성능 평가

#### 0.2.1 Airfoil Self-Noise 데이터셋
- **최적 모델**: StandardScaler + ExtraTrees (R² = 0.8496)
- **테스트 성능**: 
  - MSE: 4.69776
  - R²: 0.90163
- **특징**: 특성 변수가 5개로 비교적 단순하며, 비선형 앙상블 모델이 우수한 성능을 보였습니다.

#### 0.2.2 Concrete Compressive Strength 데이터셋
- **최적 모델**: RobustScaler + ExtraTrees (R² = 0.8842)
- **테스트 성능**: 
  - MSE: 24.45682
  - R²: 0.90851
- **특징**: 콘크리트 구성 성분 간의 복잡한 상호작용을 앙상블 트리 모델이 효과적으로 포착했습니다.

#### 0.2.3-1 Abalone 데이터셋
- **최적 모델**: MinMaxScaler + LinearRegression (R² = 0.5375)
- **테스트 성능**: 
  - MSE: 4.92557
  - R²: 0.52098
- **특징**: 위 두 데이터 셋에 비해 저조한 성능을 보였습니다. AutoML만을 사용하기 보다는 머신러닝 개발자의 개입이 필요한 상황으로 보여집니다.

#### 0.2.3-2 Abalone 데이터셋 with Feature Engineering
- **최적 모델**: RobustScaler + Ridge (R² = 0.5360)
- **테스트 성능**: 
  - MSE: 7.17803
  - R²: 0.30192
- **특징**: 특성 공학을 이용해 저조했던 이전 성능을 개선하려고 시도했지만 큰 성과는 없었습니다. 오히려 앙상블의 성능이 매우 저조했습니다.

### 0.3 성능 개선 및 효율성 향상 방안

1. **AutoML 시간 제한 확장**: 
   - 현재 120초로 제한된 탐색 시간을 늘려 더 넓은 하이퍼파라미터 공간을 탐색할 수 있습니다.

2. **전처리 파이프라인 강화**:
   - 특성 선택 및 생성 단계를 추가하여 모델 성능을 향상시킬 수 있습니다.
   - 이상치 처리를 통해 노이즈를 줄여 특히 Abalone 데이터셋의 성능을 개선할 수 있습니다.

3. **앙상블 전략 최적화**:
   - 여러 모델의 예측을 결합하여 예측 정확도를 높일 수 있습니다.
   - 스태킹 또는 블렌딩과 같은 고급 앙상블 기법을 적용할 수 있습니다.

## 1. Library Imports

In [39]:
import pandas as pd
import numpy as np
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from auto_sklearn2 import AutoSklearnRegressor
import warnings
warnings.filterwarnings("ignore")

## 2. Import Data

In [40]:
airfoil_self_noise_data = pd.read_csv(
    "airfoil_self_noise.dat",
    sep="\t",
    header=None,
    names=[
        "Frequency",
        "AngleOfAttack",
        "ChordLength",
        "FreeStreamVelocity",
        "SuctionSideDisplacement",
        "ScaledSoundLevel",
    ],
)
concrete_data = pd.read_excel("Concrete_Data.xls")
abalone_data = pd.read_csv(
    "abalone.data",
    names=[
        "Sex",
        "Length",
        "Diameter",
        "Height",
        "WholeWeight",
        "ShuckedWeight",
        "VisceraWeight",
        "ShellWeight",
        "Rings",
    ],
)

RANDOM_STATE = 2019313647
data_sets = {
    # "Airfoil": airfoil_self_noise_data,
    # "Concrete": concrete_data,
    "Abalone": abalone_data,
}

## 3. EDA

In [41]:
for name, data_set in data_sets.items():
    print("=" * 20, name, "=" * 20)
    print(data_set.shape)
    print(data_set.head())
    print(data_set.describe())

==================== Abalone ====================
(4177, 9)
  Sex  Length  Diameter  Height  WholeWeight  ShuckedWeight  VisceraWeight  \
0   M   0.455     0.365   0.095       0.5140         0.2245         0.1010   
1   M   0.350     0.265   0.090       0.2255         0.0995         0.0485   
2   F   0.530     0.420   0.135       0.6770         0.2565         0.1415   
3   M   0.440     0.365   0.125       0.5160         0.2155         0.1140   
4   I   0.330     0.255   0.080       0.2050         0.0895         0.0395   

   ShellWeight  Rings  
0        0.150     15  
1        0.070      7  
2        0.210      9  
3        0.155     10  
4        0.055      7  
            Length     Diameter       Height  WholeWeight  ShuckedWeight  \
count  4177.000000  4177.000000  4177.000000  4177.000000    4177.000000   
mean      0.523992     0.407881     0.139516     0.828742       0.359367   
std       0.120093     0.099240     0.041827     0.490389       0.221963   
min       0.075000     

In [44]:
def print_top_models(performances, n=3):
    # 딕셔너리를 성능(R²)에 따라 내림차순 정렬
    sorted_performances = sorted(performances.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\n상위 {n}개 모델:")
    for i, (model, score) in enumerate(sorted_performances[:n], 1):
        print(f"{i}. {model}: R² = {score:.4f}")

# Abalone 데이터셋을 위한 특성 공학 함수
def engineer_abalone_features(X):
    X_engineered = X.copy()
    
    X_engineered['ShellWeight_to_WholeWeight'] = X['ShellWeight'] / X['WholeWeight']
    X_engineered['ShuckedWeight_to_WholeWeight'] = X['ShuckedWeight'] / X['WholeWeight']
    X_engineered['VisceraWeight_to_WholeWeight'] = X['VisceraWeight'] / X['WholeWeight']
    
    X_engineered['Volume'] = X['Length'] * X['Diameter'] * X['Height']
    X_engineered['Density'] = X['WholeWeight'] / X_engineered['Volume']
    
    X_engineered['Length_to_Diameter'] = X['Length'] / X['Diameter']
    X_engineered['Length_to_Height'] = X['Length'] / X['Height']
    X_engineered['Diameter_to_Height'] = X['Diameter'] / X['Height']
    
    X_engineered['Length_sq'] = X['Length'] ** 2
    X_engineered['Diameter_sq'] = X['Diameter'] ** 2
    X_engineered['Height_sq'] = X['Height'] ** 2
    
    X_engineered['Shell_Shucked_Interaction'] = X['ShellWeight'] * X['ShuckedWeight']
    X_engineered['Shell_Viscera_Interaction'] = X['ShellWeight'] * X['VisceraWeight']
    
    X_engineered.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_engineered.fillna(0, inplace=True)
    return X_engineered

TRAINING_SET_SIZE = 500
TEST_SIZE = lambda y : len(y) - TRAINING_SET_SIZE
for name, data in data_sets.items():
    print("=" * 20, name, "=" * 20)
    
    # EDA
    print(data.head())
    
    # 전처리
    if name == 'Airfoil':
        X, y = data.iloc[:, :-1], data.iloc[:, -1]
    elif name == 'Concrete':
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
    elif name == 'Abalone':
        X = pd.get_dummies(data.iloc[:, :-1], columns=['Sex'], drop_first=True)
        y = data.iloc[:, -1]
        
        X = engineer_abalone_features(X)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE(y), random_state=RANDOM_STATE
    )
    
    regressor = AutoSklearnRegressor(time_limit=120)
    regressor.fit(X_train, y_train)
    
    # 예측 및 평가
    y_pred = regressor.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE: {mse:.5f}")
    print(f"R²: {r2:.5f}")
    
    # 모델 요약
    print("\n최적 모델:")
    model_performances = regressor.get_models_performance()
    print_top_models(model_performances, n=3)

==================== Abalone ====================
  Sex  Length  Diameter  Height  WholeWeight  ShuckedWeight  VisceraWeight  \
0   M   0.455     0.365   0.095       0.5140         0.2245         0.1010   
1   M   0.350     0.265   0.090       0.2255         0.0995         0.0485   
2   F   0.530     0.420   0.135       0.6770         0.2565         0.1415   
3   M   0.440     0.365   0.125       0.5160         0.2155         0.1140   
4   I   0.330     0.255   0.080       0.2050         0.0895         0.0395   

   ShellWeight  Rings  
0        0.150     15  
1        0.070      7  
2        0.210      9  
3        0.155     10  
4        0.055      7  


/home/manamana32321/skku/25-1/데과프/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/manamana32321/skku/25-1/데과프/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/manamana32321/skku/25-1/데과프/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/manamana32321/skku/25-1/데과프/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't 

MSE: 7.17803
R²: 0.30192

최적 모델:

상위 3개 모델:
1. robust_scaler_ridge: R² = 0.5360
2. robust_scaler_poisson: R² = 0.5315
3. standard_scaler_extra_trees: R² = 0.5315


/home/manamana32321/skku/25-1/데과프/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
